In [3]:
from fpylll import IntegerMatrix
import numpy as np

In [3]:
# find l^2-SVP of lattice given an oracle to find l-HSVP
def SVP (B):
    if(B.shape[1] == 1):
        return np.linalg.norm(B) #if there is just one basis vector
    
    X = HSVP(B)
    D = B.dot(np.linalg.inv(B.dot(B.T))) #dual lattice
    Y = HSVP(D)
    L0 = sublatticeOrthogonalToGivenVector(D, Y)

    return min(np.linalg.norm(X), SVP(L0))

In [ ]:
def HSVP (B):
    #return the HSVP vector for the given lattice
    ""

In [1]:
# return sublattice along the hyperplane orthogonal to Y
def sublatticeOrthogonalToGivenVector(D, Y):
    # vector which satifies DV=Y
    V = np.linalg.solve(D, Y)

    # this will store the unimodular matrix 
    # which when multiplied with D given a lattice basis which has first vector as Y
    U = np.zeros(np.shape(D))
    U[:,0] = V

    # dimension of U
    n = U.shape[0]

    # this will store the elementary operations matrix
    Elem = np.identity(n)

    # swapping first and last columns and transposing
    U = U.dot(swap(0, -1, n))
    U_T = U.T

    # columnGCD operations for all consecutive columns
    for i in range(n-1):
        [U, Elem] = columnGCD(i, i+1, U, Elem)

    # inverting all the elementary operations
    # this creates a unimodular matrix whose first column has values 
    # such that on multiplying with D (dual of original) it will give 
    # a basis whose first column is Y
    U = U.dot(np.linalg.inv(Elem))
    U = U.T
    U = U.dot(swap(0, -1, n))

    # lattice basis of dual lattice D with first basis vector as Y
    # this has dimensions n x n-1
    M = D.dot(U)
    M = M.dot(np.linalg.inv(M.dot(M.T)))[:, 1:]

    # reduce last row as in HNF to make M a square matrix
    for i in range(n-2):
        [M, _] = columnGCD(i, i+1, M, np.identity(n-1))

    # remove last row and return
    return M[:-1, :]




In [2]:
# perform gcd operations on the last row elemets of the given vectors
# and return the matrix for elementary operations involved
def columnGCD(i, j, U, Elem):
    a = U[-1][i]
    b = U[-1][j]

    if(a==0):
        return [U, Elem]
    
    if(a>b):
        Elem = Elem.dot(swap(i, j, U.shape[0]))
        [U, Elem] = columnGCD(i, j, U.dot(Elem), Elem)
    else :
        Elem = Elem.dot(add(j, i, (-1)*(b/a), U.shape[0]))
        [U, Elem] = columnGCD(i, j, U.dot(Elem), Elem)

    return [U, Elem]


In [ ]:
def swap(i, j, n):
    I = np.identity(n)
    I[i][i] = I[j][j] = 0
    I[i][j] = I[j][i] = 1

    return I

In [ ]:
def invert(i, n):
    I = np.identity(n)
    I[i][i] = -1

    return I

In [1]:
def add(i, j, c, n):
    I = np.identity(n)
    I[i][j] = c

    return I